In [1]:
pip install pdfplumber pandas openpyxl

Defaulting to user installation because normal site-packages is not writeable
     ---------------------------------------- 0.0/43.6 kB ? eta -:--:--
     --------- ------------------------------ 10.2/43.6 kB ? eta -:--:--
     -------------------------- ----------- 30.7/43.6 kB 262.6 kB/s eta 0:00:01
     -------------------------------------- 43.6/43.6 kB 306.0 kB/s eta 0:00:00
     ---------------------------------------- 0.0/68.1 kB ? eta -:--:--
     ------------------------ --------------- 41.0/68.1 kB 2.0 MB/s eta 0:00:01
     -------------------------------------- 68.1/68.1 kB 933.2 kB/s eta 0:00:00
   ---------------------------------------- 0.0/60.0 kB ? eta -:--:--
   ------ --------------------------------- 10.2/60.0 kB ? eta -:--:--
   --------------------------- ------------ 41.0/60.0 kB 487.6 kB/s eta 0:00:01
   ---------------------------------------- 60.0/60.0 kB 638.5 kB/s eta 0:00:00
   ---------------------------------------- 0.0/6.6 MB ? eta -:--:--
   ------------

  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.


In [2]:
import pdfplumber
import pandas as pd
import re


PDF_PATH = "Aahar 2025 Fair Guide.pdf"
OUTPUT_EXCEL = "Aahar_2025_Extracted_Data.xlsx"


def extract_text_from_pdf(pdf_path):
    """Extract full text from PDF"""
    full_text = ""
    with pdfplumber.open(pdf_path) as pdf:
        for page in pdf.pages:
            text = page.extract_text()
            if text:
                full_text += text + "\n"
    return full_text


def parse_companies(text):
    """
    Parse company details using regex.
    Assumption: Each company block is separated clearly in PDF.
    """
    records = []

    blocks = re.split(r"\n\s*\n", text)  # split by empty lines

    for block in blocks:
        company = re.search(r"Company Name\s*:\s*(.*)", block, re.I)
        address = re.search(r"Address\s*:\s*(.*)", block, re.I)
        contact = re.search(r"Contact Person\s*:\s*(.*)", block, re.I)
        phone = re.search(r"(Tel\.?|Mobile)\s*:\s*(.*)", block, re.I)
        email = re.search(r"Email\s*:\s*(.*)", block, re.I)

        if company:
            records.append({
                "Company Name": company.group(1).strip() if company else "",
                "Address": address.group(1).strip() if address else "",
                "Contact Person": contact.group(1).strip() if contact else "",
                "Tel / Mobile": phone.group(2).strip() if phone else "",
                "Email": email.group(1).strip() if email else ""
            })

    return records


def main():
    print("📄 Extracting text from PDF...")
    text = extract_text_from_pdf(PDF_PATH)

    print("🔍 Parsing company data...")
    data = parse_companies(text)

    df = pd.DataFrame(data)

    print("📊 Saving to Excel...")
    df.to_excel(OUTPUT_EXCEL, index=False)

    print("✅ Done! File saved as:", OUTPUT_EXCEL)


if __name__ == "__main__":
    main()

📄 Extracting text from PDF...
🔍 Parsing company data...
📊 Saving to Excel...
✅ Done! File saved as: Aahar_2025_Extracted_Data.xlsx


In [3]:
import pdfplumber
import pandas as pd
import re

PDF_PATH = "Aahar 2025 Fair Guide.pdf"
OUTPUT_EXCEL = "Aahar_2025_Extracted_Data.xlsx"


def extract_records(pdf_path):
    records = []

    with pdfplumber.open(pdf_path) as pdf:
        for page in pdf.pages:
            text = page.extract_text()
            if not text:
                continue

            lines = [l.strip() for l in text.split("\n") if l.strip()]

            for i, line in enumerate(lines):
                # Email detection
                email_match = re.search(r"[A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\.[A-Za-z]{2,}", line)

                if email_match:
                    email = email_match.group()
                    company = lines[i - 1] if i > 0 else ""

                    phone = ""
                    for j in range(i, min(i + 3, len(lines))):
                        phone_match = re.search(r"(\+?\d[\d\s-]{7,})", lines[j])
                        if phone_match:
                            phone = phone_match.group()
                            break

                    records.append({
                        "Company Name": company,
                        "Address": "",
                        "Contact Person": "",
                        "Tel / Mobile": phone,
                        "Email": email
                    })

    return records


def main():
    print("📄 Extracting data from PDF...")
    data = extract_records(PDF_PATH)

    df = pd.DataFrame(data)
    df.drop_duplicates(inplace=True)

    df.to_excel(OUTPUT_EXCEL, index=False)
    print(f"✅ Data extracted successfully: {OUTPUT_EXCEL}")


if __name__ == "__main__":
    main()

📄 Extracting data from PDF...
✅ Data extracted successfully: Aahar_2025_Extracted_Data.xlsx
